In [ ]:
import pygui as pgui
pgui.create_interface()


# Example usage:

        # F =>  String: Faction                  'Alloyn'              F=Alloyn
        # D =>  String: Deckname                 'Insane'              D~Forge
        # FB => String: Forgeborn               'Ironbeard'            FB=Ironbeard
        # C  => List:   Cardnames               'Digitize'             C~Digitize
        # A  => List:   Forgeborn - Ability     'Army Commander'       A~'Army Commander'
        # K  => Dict:   Composition             'Robot'                K:Robot > 3

In [7]:
import pygui as pgui
import pandas as pd
import buckaroo  
import qgrid
from IPython.display import display
from MongoDB.DatabaseManager import DatabaseManager
import GlobalVariables
from icecream import ic

def qgrid_show(df):
    grid_options = {
    'fullWidthRows': True,
    'syncColumnCellResize': True,
    'forceFitColumns': True,
    'defaultColumnWidth': 200,
    'rowHeight': 28,
    'enableColumnReorder': True,
    'enableTextSelectionOnCells': True,
    'editable': False,
    'autoEdit': False,
    'explicitInitialization': True,
    'maxVisibleRows': 15,
    'minVisibleRows': 8,
    'sortable': True,
    'filterable': True,
    'highlightSelectedCell': False,
    'highlightSelectedRow': True
    }    

    col_options =           { 'width': 50, }
    col_defs = {        
        'name':             { 'width': 250, },
        'registeredDate':   { 'width': 200, },
        'cardSetNo':        { 'width': 50,  },
        'faction':          { 'width': 100,  },
        'forgebornId':      { 'width': 100,  },
    }

    qgrid_df = qgrid.show_grid(df,
                               column_options=col_options,
                               column_definitions=col_defs,
                               grid_options={'forceFitColumns': False},
                               show_toolbar=True)
    display(qgrid_df)

df = pgui.collection_to_df('Deck')

GlobalVariables.username = 'tmind'
myDB = DatabaseManager(GlobalVariables.username)
commonDB = DatabaseManager('common')
db_name = myDB.get_current_db_name()

# Get all Decks from the database
deck_cursor = myDB.find('Deck', {})
df_decks = pd.DataFrame(list(deck_cursor))
df_decks_filtered = df_decks[[ 'registeredDate', 'name', 'cardSetNo', 'faction', 'forgebornId']].copy()

additional_columns = ['A1', 'A2', 'A3', 'H1', 'H2', 'H3']
for column in additional_columns:
    df_decks_filtered.loc[:,column] = 0.0

df_decks_filtered.set_index('name', inplace=True)

# Create a DataFrame from the 'stats' sub-dictionary

for deck in myDB.find('Deck', {}):
    if 'stats' in deck:
        
        stats = deck.get('stats', {})
        # Convert the 'card_types' sub-dictionary into a DataFrame
        card_types_df = pd.DataFrame.from_dict(stats['card_types'], orient='index')
        creature_averages_df = pd.DataFrame.from_dict(stats['creature_averages'], orient='index')
           
        # Flatten the 'creature_averages' sub-dictionaries into a single-row DataFrame
        # Convert the dictionary into a DataFrame
        attack_dict = stats['creature_averages']['attack']
        attack_df = pd.DataFrame([attack_dict], columns=attack_dict.keys())
        attack_df.columns = ['A1', 'A2', 'A3']

        defense_dict = stats['creature_averages']['health']
        defense_df = pd.DataFrame([defense_dict], columns=defense_dict.keys())
        defense_df.columns = ['H1', 'H2', 'H3']
        
        # Combine the attack and defense DataFrames into a single DataFrame
        deck_stats_df = pd.concat([attack_df, defense_df], axis=1)
        deck_stats_df['name'] = deck['name']
        
        # Round each value in the DataFrame
        deck_stats_df = deck_stats_df.round(2)
                                        
        # Set the common column as the index in both dataframes
        deck_stats_df.set_index('name', inplace=True)

        # Update the corresponding row in df_decks_filtered with the stats from deck_stats_df
        df_decks_filtered.update(deck_stats_df)

    # Replace forgebornId with the forgeborn name from the database     
    forgeborn = commonDB.find('Forgeborn', {'_id': deck['forgebornId']})
    
    if forgeborn:        
        df_decks_filtered.loc[deck['name'], 'forgebornId'] = forgeborn[0]['name']
    
# Reset the index in df_decks_filtered
df_decks_filtered.reset_index(inplace=True)
qgrid_show(df_decks_filtered)  



QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defa…

In [ ]:
import pandas as pd
import buckaroo
pd.DataFrame({'a':[1, 2, 10, 30, 50, 60, 50], 'b': ['foo', 'foo', 'bar', pd.NA, pd.NA, pd.NA, pd. NA]})


In [ ]:
buckaroo.disable()